In [12]:
#  Ini full manual ################

import math
from datetime import datetime

def calculate_julian_date(year, month, day, hour=0, minute=0, second=0):
    # Jika bulan Januari atau Februari, anggap sebagai bagian dari tahun sebelumnya
    if month <= 2:
        year -= 1
        month += 12
    A = math.floor(year / 100)
    B = 2 - A + math.floor(A / 4)
    JD = math.floor(365.25 * (year + 4716)) + math.floor(30.6001 * (month + 1)) + day + B - 1524.5
    JD += (hour + minute / 60 + second / 3600) / 24.0  # Tambahkan waktu dalam hari
    return JD

def centuries_since_epoch(JD):
    # Menghitung jumlah abad Julian sejak epoch 2000.0
    return (JD - 2451545.0) / 36525.0

def obliquity_of_ecliptic(T):
    # Approximate mean obliquity of the ecliptic
    ε = 23.439292 - 0.000013 * T
    return math.radians(ε)

def ecliptic_to_equatorial(λ, β, ε):
    # Convert ecliptic (λ, β) to equatorial (α, δ)
    λ, β = math.radians(λ), math.radians(β)
    α = math.atan2(math.sin(λ) * math.cos(ε) - math.tan(β) * math.sin(ε), math.cos(λ))
    δ = math.asin(math.sin(β) * math.cos(ε) + math.cos(β) * math.sin(ε) * math.sin(λ))
    return math.degrees(α) % 360, math.degrees(δ)

def local_sidereal_time(julian_day, longitude):
    # Compute the Local Sidereal Time (LST)
    JD0 = math.floor(julian_day - 0.5) + 0.5
    H = (julian_day - JD0) * 24.0
    D = julian_day - 2451545.0
    GMST = 18.697374558 + 24.06570982441908 * D + H
    GMST %= 24
    LST = (GMST + longitude / 15.0) % 24  # Convert longitude to hours
    return LST * 15  # Convert to degrees

def equatorial_to_horizontal(α, δ, LST, latitude):
    # Convert equatorial (α, δ) to horizontal (Alt, Az)
    α, δ = math.radians(α), math.radians(δ)
    H = math.radians((LST - α * 180 / math.pi) % 360)  # Hour angle in radians
    latitude = math.radians(latitude)

    alt = math.asin(math.sin(δ) * math.sin(latitude) + math.cos(δ) * math.cos(latitude) * math.cos(H))
    az = math.atan2(-math.cos(δ) * math.cos(latitude) * math.sin(H),
                    math.sin(δ) - math.sin(alt) * math.sin(latitude))
    alt, az = math.degrees(alt), math.degrees(az) % 360
    return alt, az

# Input data
year, month, day = 2024, 12, 12
hour, minute, second = 20, 56, 0
longitude = 112 + 46 / 60 + 56.9 / 3600  # Convert longitude to decimal degrees
latitude = -(7 + 16 / 60 + 11.4 / 3600)  # Convert latitude to decimal degrees

# Compute Julian Date and centuries since epoch
JD = calculate_julian_date(year, month, day, hour, minute, second)
T = centuries_since_epoch(JD)

# Example ecliptic coordinates of the Moon (replace λ, β with real values)
λ = 136.46  # Ecliptic longitude in degrees
β = -5.10   # Ecliptic latitude in degrees

# Compute obliquity and convert
ε = obliquity_of_ecliptic(T)
α, δ = ecliptic_to_equatorial(λ, β, ε)

# Compute Local Sidereal Time
LST = local_sidereal_time(JD, longitude)

# Convert to horizontal coordinates
alt, az = equatorial_to_horizontal(α, δ, LST, latitude)

# Display results
print(f"Julian Date: {JD:.6f}")
print(f"Right Ascension (α): {α:.6f}°")
print(f"Declination (δ): {δ:.6f}°")
print(f"Altitude: {alt:.6f}°")
print(f"Azimuth: {az:.6f}°")


Julian Date: 2460657.372222
Right Ascension (α): 137.359587°
Declination (δ): 11.032708°
Altitude: 51.066805°
Azimuth: 62.295944°


In [15]:
#  Ini full library #####################

from skyfield.api import Topos, load

# Lokasi pengamatan
latitude = -(7 + 16 / 60 + 11.4 / 3600)  # -7°16'11.4"S
longitude = 112 + 46 / 60 + 56.9 / 3600  # 112°46'56.9"E
location = Topos(latitude_degrees=latitude, longitude_degrees=longitude)

# Waktu pengamatan
ts = load.timescale()
t = ts.utc(2024, 12, 12, 13, 56, 0)  # Waktu dalam UTC (WIB = UTC+7)

# Data efemeris
eph = load('de440.bsp')  # Ephemeris data
earth = eph['earth']
moon = eph['moon']

# Hitung posisi
targetPos = earth + location  # Lokasi pengamat di permukaan Bumi
apparent = targetPos.at(t).observe(moon).apparent()
alt, az, _ = apparent.altaz()

print(f"Altitude: {alt.degrees:.2f}°")
print(f"Azimuth: {az.degrees:.2f}°")


Altitude: 63.10°
Azimuth: 352.15°


In [ ]:
# ini jadi #################################

from skyfield.api import Topos, load
from math import sin, cos, asin, atan2, radians, degrees

# Lokasi pengamatan
latitude = -(7 + 16 / 60 + 11.4 / 3600)  # -7°16'11.4"S
longitude = 112 + 46 / 60 + 56.9 / 3600  # 112°46'56.9"E
location = Topos(latitude_degrees=latitude, longitude_degrees=longitude)

# Waktu pengamatan
ts = load.timescale()
t = ts.utc(2185, 11, 30, 16, 52, 48)  # WIB (UTC+7), jadi masukkan dalam UTC

# Data efemeris
eph = load('de440.bsp')
moon = eph['moon']
earth = eph['earth']

# Hitung koordinat ekuatorial (RA, Dec)
observerPos = earth + location
targetPos = observerPos.at(t).observe(moon).apparent()
ra, dec, _ = targetPos.radec()  # Right ascension dan declination

# Konversi ke koordinat horizontal secara manual
# Langkah-langkah:
def equatorial_to_horizontal(ra, dec, latitude, longitude, time):
    """Konversi dari ekuatorial ke horizontal (altitude dan azimuth)."""
    # Local Sidereal Time (LST)
    lst = (time.gast + longitude / 15.0) % 24  # dalam jam
    lst_rad = radians(lst * 15)  # konversi ke radian

    # RA dan Dec dalam radian
    ra_rad = ra.radians
    dec_rad = dec.radians

    # Latitude dalam radian
    lat_rad = radians(latitude)

    # Hour Angle (HA)
    ha_rad = lst_rad - ra_rad

    # Altitude
    alt_rad = asin(
        sin(lat_rad) * sin(dec_rad) +
        cos(lat_rad) * cos(dec_rad) * cos(ha_rad)
    )

    # Azimuth
    az_rad = atan2(
        -cos(dec_rad) * sin(ha_rad),
        sin(dec_rad) - sin(lat_rad) * sin(alt_rad)
    )

    # Konversi ke derajat
    altitude = degrees(alt_rad)
    azimuth = (degrees(az_rad) + 360) % 360  # pastikan azimuth dalam rentang [0, 360)

    return altitude, azimuth

# Hitung Altitude dan Azimuth
altitude, azimuth = equatorial_to_horizontal(
    ra, dec, latitude, longitude, t
)

print(f"Altitude: {altitude:.2f}°")
print(f"Azimuth: {azimuth:.2f}°")
print(eph)

Altitude: -1.59°
Azimuth: 259.69°
SPICE kernel file 'de440.bsp' has 14 segments
  JD 2287184.50 - JD 2688976.50  (1549-12-30 through 2650-01-24)
      0 -> 1    SOLAR SYSTEM BARYCENTER -> MERCURY BARYCENTER
      0 -> 2    SOLAR SYSTEM BARYCENTER -> VENUS BARYCENTER
      0 -> 3    SOLAR SYSTEM BARYCENTER -> EARTH BARYCENTER
      0 -> 4    SOLAR SYSTEM BARYCENTER -> MARS BARYCENTER
      0 -> 5    SOLAR SYSTEM BARYCENTER -> JUPITER BARYCENTER
      0 -> 6    SOLAR SYSTEM BARYCENTER -> SATURN BARYCENTER
      0 -> 7    SOLAR SYSTEM BARYCENTER -> URANUS BARYCENTER
      0 -> 8    SOLAR SYSTEM BARYCENTER -> NEPTUNE BARYCENTER
      0 -> 9    SOLAR SYSTEM BARYCENTER -> PLUTO BARYCENTER
      0 -> 10   SOLAR SYSTEM BARYCENTER -> SUN
      3 -> 301  EARTH BARYCENTER -> MOON
      3 -> 399  EARTH BARYCENTER -> EARTH
      1 -> 199  MERCURY BARYCENTER -> MERCURY
      2 -> 299  VENUS BARYCENTER -> VENUS


In [114]:
# ini convert eq to horizon #################################

from skyfield.api import Topos, load
from math import sin, cos, asin, atan2, radians, degrees
import re

## Lokasi. contoh input: lats = 6°52'08.2"S | longs = 112°19'56.4"E
# lats = re.sub(r'[°\'"]', ' ', input('Latitude: ')).split()
# longs = re.sub(r'[°\'"]', ' ', input('Longitude: ')).split()
lats = ['7', '16', '11.4', 'S']
longs = ['112', '46', '56.9', 'E']
# print(lats, longs)

# Concate koordinat, ignore kardinal
latDeg, latArcmin, latArcsec = map(float, lats[:-1])
longDeg, longArcmin, longArcsec = map(float, longs[:-1])

# Konversi ke desimal
latitude = latDeg + latArcmin / 60 + latArcsec / 3600
longitude = longDeg + longArcmin / 60 + longArcsec / 3600

# Tandai negatif jika lintang LS atau bujur BB
if lats[-1] == 'S':
    latitude = -(latitude)
if longs[-1] == 'W':
    longitude = -(longitude)

# convert koordinat geografis (bola) jadi vektor geosentris
location = Topos(latitude_degrees=latitude, longitude_degrees=longitude)
print(location)

## Waktu pengamatan

# d, mon, y, h, mins, sec = map(float, (re.sub(r'[-:,]', ' ', input('Time (dd-mm-yyy,hh:mm:ss) ')).split()))
d, mon, y, h, mins, sec = 12, 12, 2024, 20, 56, 16
h -= longitude//15 # convert jam ke UTC

ts = load.timescale()
t = ts.utc(y, mon, d, h, mins, sec)

# Data efemeris
eph = load('de421.bsp')
moon = eph['moon']
earth = eph['earth']

# Hitung koordinat ekuatorial (RA, Dec)
observerPos = earth + location
targetPos = observerPos.at(t).observe(moon).apparent()
ra, dec, d = targetPos.radec()  # Right ascension dan declination

# Konversi ke koordinat horizontal secara manual
def equatorial_to_horizontal(ra, dec, latitude, longitude, time):
    """Konversi dari ekuatorial ke horizontal (altitude dan azimuth)."""
    # Local Sidereal Time (LST)
    lst = (time.gast + longitude / 15.0) % 24  # dalam jam
    lst_rad = radians(lst * 15)  # konversi ke radian

    # RA dan Dec dalam radian
    ra_rad = ra.radians
    dec_rad = dec.radians

    # Latitude dalam radian
    lat_rad = radians(latitude)

    # Hour Angle (HA)
    ha_rad = lst_rad - ra_rad

    # Altitude
    alt_rad = asin(
        sin(lat_rad) * sin(dec_rad) +
        cos(lat_rad) * cos(dec_rad) * cos(ha_rad)
    )

    # Azimuth
    az_rad = atan2(
        -cos(dec_rad) * sin(ha_rad),
        sin(dec_rad) - sin(lat_rad) * sin(alt_rad)
    )

    # Konversi ke derajat
    altitude = degrees(alt_rad)
    azimuth = (degrees(az_rad) + 360) % 360  # pastikan azimuth dalam rentang [0, 360)

    return altitude, azimuth

# Hitung Altitude dan Azimuth
altitude, azimuth = equatorial_to_horizontal(
    ra, dec, latitude, longitude, t
)

print(f"Altitude: {altitude:.2f}°")
print(f"Azimuth: {azimuth:.2f}°")


IERS2010 latitude -7.2698 N longitude 112.7825 E elevation 0.0 m
Altitude: 63.15°
Azimuth: 351.19°


In [ ]:
# transform vektor ke altazimuth
def vec_altz(x, y, z, decimal=False):
    latitudeRad = np.radians(latitude)
    lstRad = np.radians(lst)

    # convert vektor barycentric ke vektor horizontal
    matrixTransform = np.array([
        [-np.sin(lstRad), np.cos(lstRad), 0],
        [-np.sin(latitudeRad) * np.cos(lstRad), -np.sin(latitudeRad) * np.sin(lstRad), np.cos(latitudeRad)],
        [np.cos(latitudeRad) * np.cos(lstRad), np.cos(latitudeRad) * np.sin(lstRad), np.sin(latitudeRad)]
    ])

    xyz = np.array([x, y, z])
    xH, yH, zH = matrixTransform.dot(xyz)
    
    alt = np.degrees(np.asin(zH))       # sin Alt = zH
    az = np.degrees(np.atan2(yH, xH))   # tan Az = yH/xH
    if az < 0:
        az += 360

    if decimal == False:
        return celesCoor(alt), celesCoor(az)

    return alt, az

In [ ]:
# convert vektor geosentris to eq to horizon #################################

from skyfield.api import Topos, load
import numpy as np
import re

# Format celestial coordinate ke seksagesimal (basis 60)
def celesCoor(decimal, hour=False, pnt=2):
    unit = ['°', "'", '"']
    # convert derajat jadi jam (15° = 1h)
    if hour:
        mainComp_h = decimal/15
        decimal = mainComp_h
        unit = ['h', 'm', 's']
    # kalo seksagesimal derajat:
    mainComp = int(decimal)
    min_float = (decimal-mainComp)*60
    min = int(min_float)
    sec = (min_float - min)*60

    return f'{mainComp}{unit[0]}{min}{unit[1]}{sec:.{pnt}f}{unit[2]}'

# cari vektor bulan (x,y,z,r) terhadap koordinat geografis observer
def vectorFrom(object, lat, long, time):
    # Data efemeris posisi
    eph = load('de421.bsp')
    earth = eph['earth']
    surfaceLoc = Topos(latitude=lat, longitude=long)            # posisi observer (lat, long, elevation)
    observerPos = earth + surfaceLoc                            # vektor observer -> earth barycenter (x, y, z)
    targetPos = observerPos.at(time).observe(eph[object]).apparent()  # posisi target terhadap observer di permukaan
    x, y, z = targetPos.position.au                             # satuan AU diitung dari permukaan bumi
    r = np.sqrt(x**2 + y**2 + z**2)                             # panjang vektor (jarak)
    return x, y, z, r

# transform vektor ke equatorial
def vec_eq(x, y, z, r, decimal=False):
    # Format desimal
    dec = np.degrees(np.asin(z/r))      # sin δ = z/r
    ra = np.degrees(np.atan2(y, x))     # tan α = y/x
    # Make sure RA di range 0 - 360°
    if ra < 0:
        ra += 360
    # Format celestial coordinate
    if decimal == False:
        return celesCoor(dec), celesCoor(ra, hour=True)
    return ra, dec

def HA(lst, ra):
    lst = np.radians(lst*15)
    ra = np.radians(ra)
    
    ha = np.degrees(lst - ra)
    if ha < 0:
        ha += 360
    return ha

def eq_altz(lat, dec, ra, ha):
    lat = np.radians(lat)
    dec = np.radians(dec)
    ha = np.radians(ha)
    ra = np.radians(ra)

    alt = np.asin((np.sin(dec) * np.sin(lat)) + (np.cos(dec) * np.cos(lat) * np.cos(ha)))
    # az = np.asin(np.sin(ha) * np.cos(dec) / np.cos(ra))
    az = np.atan2(-np.cos(dec) * np.sin(ha), np.sin(dec) - np.sin(lat) * np.sin(alt))
    az = (np.degrees(az) + 360) % 360

    return celesCoor(np.degrees(alt)), celesCoor(az)


## Lokasi. contoh input: lats = 6°52'08.2"S | longs = 112°19'56.4"E
# lats = re.sub(r'[°\'"]', ' ', input('Latitude: ')).split()
# longs = re.sub(r'[°\'"]', ' ', input('Longitude: ')).split()
lats = ['7', '16', '11', 'S']
longs = ['112', '46', '57', 'E']
# print(lats, longs)

# Concate koordinat, ignore kardinal
latDeg, latArcmin, latArcsec = map(float, lats[:-1])
longDeg, longArcmin, longArcsec = map(float, longs[:-1])

# Konversi ke desimal
latitude = (latDeg + latArcmin / 60 + latArcsec / 3600)
longitude = (longDeg + longArcmin / 60 + longArcsec / 3600)

# Tandai negatif jika lintang LS atau bujur BB
if lats[-1] == 'S':
    latitude = -(latitude)
if longs[-1] == 'W':
    longitude = -(longitude)

## Waktu pengamatan
# d, mon, y, h, mins, sec = map(float, (re.sub(r'[-:,]', ' ', input('Time (dd-mm-yyy,hh:mm:ss) ')).split()))
d, mon, yr, h, mins, sec = 12, 12, 2024, 20, 56, 0
h -= longitude//15 # convert jam ke UTC

# target = input('Target of observation : ')
target = 'moon'

ts = load.timescale()
t = ts.utc(yr, mon, d, h, mins, sec)
lst = (t.gast + longitude / 15) % 24  # dalam jam

# Transformasi vektor -> RA, dec -> Alt, Az
x, y, z, r = vectorFrom(target, latitude, longitude, t)
ra, dec = vec_eq(x, y, z, r, True)
ha = HA(lst, ra)
alt, az = eq_altz(latitude, dec, ra, ha)

ra_sx, dec_sx = vec_eq(x, y, z, r)

# print(x,y,z,r)
# print(ra_sx, dec_sx)
# print(alt, az)

print('target : ', target, flush=True)
print('Lat  :', latitude, flush=True)
print('Long  :', longitude, flush=True)
print('vector:', x, y, z, r, flush=True)
print('time  :', t, flush=True)
print('LST  :', lst, flush=True)
print('HA  :', ha, flush=True)
print('RA  :', ra, flush=True)
print('RA frmt:', ra_sx, flush=True)
print('Dec  :', dec, flush=True)
print('Dec frmt :', dec_sx, flush=True)
print('Alt  :', alt, flush=True)
print('Az   :', az, flush=True)


target :  moon
Lat  : -7.269722222222222
Long  : 112.7825
vector: 0.0017529804828181966 0.001441796372698572 0.0007931292011790224 0.002404323456449059
time  : <Time tt=2460657.0813562963>
LST  : 2.9027380384125934
HA  : 4.10436016440291
RA  : 39.43671041178599
RA frmt: 19°15'40.55"
Dec  : 19.261264433376226
Dec frmt : 2h37m44.81s
Alt  : 63°9'45.34"
Az   : 351°19'26.03"


In [ ]:
import datetime

ts = load.timescale()
t = ts.utc(2024, 12, 12, 20, 56)
tm = t.utc_datetime()
print(t)
print(tm)
formatted = tm.strftime()


<Time tt=2460657.373022963>
2024-12-12 20:56:00+00:00
